# Import lines

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import time
import random

from pandas import *

# SK-learn libraries.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import re

# scipy
from scipy.signal import butter, lfilter


Merge the test data into a single csv.

In [15]:
#Remove the channels we don't want 
def Remove_Channels(df):
    df.drop(df.columns[[15,16,20,21,22,25,26,27,28,29,30,31,32]], axis=1, inplace=True)
    return df



path =r'C:\Users\marks\Google Drive\EEG Kaggle\train'
train_data_filenames = glob.glob(path + "/subj1_series[1-2]*data.csv") #load only subject 1, series 1-3
list_ = []
Train_Array_Lengths = []
Start_Time = time.time()
print time.time(), "Merging..."

for file_ in train_data_filenames:
    print time.time(), file_
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(Remove_Channels(df))
    Train_Array_Lengths.append(len(df))
train_data = pd.concat(list_)

print "Merge Complete:", time.time()-Start_Time, "total seconds"




#now with event data
path =r'C:\Users\marks\Google Drive\EEG Kaggle\train'
train_event_filenames = glob.glob(path + "/subj1_series[1-2]*events.csv") #load only subject 1, series 1-3
list_ = []

Start_Time = time.time()
print time.time(), "Merging..."

for file_ in train_event_filenames:
    print time.time(), file_
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
train_events = pd.concat(list_)

print "Merge Complete:", time.time()-Start_Time, "total seconds"




#do the same thing and now get your testing data
test_data_filenames = glob.glob(path + "/subj1_series3*data.csv") #load only subject 1, series 1-3
list_ = []
Test_Array_Lengths = []
Start_Time = time.time()
print time.time(), "Merging..."

for file_ in test_data_filenames:
    print time.time(), file_
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(Remove_Channels(df))
    Test_Array_Lengths.append(len(df))
test_data = pd.concat(list_)

print "Merge Complete:", time.time()-Start_Time, "total seconds"




#now with event data
path =r'C:\Users\marks\Google Drive\EEG Kaggle\train'
test_event_filenames = glob.glob(path + "/subj1_series3*events.csv") #load only subject 1, series 1-3
list_ = []

Start_Time = time.time()
print time.time(), "Merging..."

for file_ in test_event_filenames:
    print time.time(), file_
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
test_events = pd.concat(list_)

print "Merge Complete:", time.time()-Start_Time, "total seconds"

1438711235.99 Merging...
1438711235.99 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series1_data.csv
1438711237.17 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series2_data.csv
Merge Complete: 2.69600009918 total seconds
1438711238.69 Merging...
1438711238.69 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series1_events.csv
1438711238.85 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series2_events.csv
Merge Complete: 0.512000083923 total seconds
1438711239.21 Merging...
1438711239.21 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series3_data.csv
Merge Complete: 1.16199994087 total seconds
1438711240.38 Merging...
1438711240.38 C:\Users\marks\Google Drive\EEG Kaggle\train\subj1_series3_events.csv
Merge Complete: 0.266999959946 total seconds


Train a simple model and make predictions on the test data.

In [16]:
print test_data.head(10)

                id  Fp1  Fp2   F7   F3  Fz   F4   F8  FC5  FC1  FC2  FC6   T7  \
0  subj1_series3_0   16  274  243  240 -80  410 -731  466  310   93  319  385   
1  subj1_series3_1   26  255  266  203 -93  408 -770  487  296   57  312  735   
2  subj1_series3_2   65  206  300  221 -77  416 -813  538  316   64  254  668   
3  subj1_series3_3   81  203  343  183 -73  428 -766  467  283   66  217  499   
4  subj1_series3_4   97  259  382  160 -61  441 -653  480  287   78  271  535   
5  subj1_series3_5   79  259  328  154 -51  417 -682  465  297   85  321  634   
6  subj1_series3_6   13  209  310   97 -56  374 -727  443  298  100  291  649   
7  subj1_series3_7  -22  164  328  107 -72  421 -747  437  302  107  312  566   
8  subj1_series3_8  -66  172  297  157 -79  452 -677  433  301  102  371  369   
9  subj1_series3_9   32  262  300  172 -50  514 -537  422  299  106  476  426   

    C3  Cz  TP9  CP5  CP1   P7   P3  
0  237 -30 -285  204   48  407  230  
1  240 -67 -336  237   37  448  

In [3]:
Train_Labels_HandStart =  train_events['HandStart']
Train_Labels_FirstDigitTouch =  train_events['FirstDigitTouch']
Train_Labels_BothStartLoadPhase =  train_events['BothStartLoadPhase']
Train_Labels_LiftOff =  train_events['LiftOff']
Train_Labels_Replace =  train_events['Replace']
Train_Labels_BothReleased =  train_events['BothReleased']
# Train_Labels_Combined = train_events.HandStart.map(str) + ',' + train_events.FirstDigitTouch.map(str) + ',' + train_events.BothStartLoadPhase.map(str) + ',' + train_events.LiftOff.map(str) + ',' + train_events.Replace.map(str) + ',' + train_events.BothReleased.map(str)


Train_Data = train_data.iloc[0:,1:] # select only column data


Test_Labels_HandStart =  test_events['HandStart']
Test_Labels_FirstDigitTouch =  test_events['FirstDigitTouch']
Test_Labels_BothStartLoadPhase =  test_events['BothStartLoadPhase']
Test_Labels_LiftOff =  test_events['LiftOff']
Test_Labels_Replace =  test_events['Replace']
Test_Labels_BothReleased =  test_events['BothReleased']


Test_Data = test_data.iloc[0:,1:] # select only data columns





Define functions for feature extraction

In [4]:

#bin the time 
def Bin_Time(num_rows,num_bins):
    Bin_Size = num_rows/num_bins
    Bins = np.zeros(shape=(num_rows,num_bins))
    Bin_Min = 0
    Bin_Max = Bin_Size
    for i in range(0,num_bins):
        Bins[Bin_Min:Bin_Max,i] = 1
        Bin_Min = Bin_Min + Bin_Size
        Bin_Max = Bin_Max + Bin_Size
    return Bins


##filters borrowed from Nihar. 
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


#run PCA and return the number of PCs that explain the given amount of variance. 
def extractFeatures_PCA(df,Test_df, PercentVarExplained):
    data = df.as_matrix()
    print len(data)
    test_data = Test_df.as_matrix()
    pca = PCA()
    pca.fit(data)  
    
    Explained_Variance_Ratios = pca.explained_variance_ratio_
    for i in range(1,len(Explained_Variance_Ratios)):
        if sum(Explained_Variance_Ratios[0:i]) >= PercentVarExplained:
                   NumPCs = i + 1 #add 1 since numpy array ranges are not inclusive
                   break
    PCA_Projections = pca.transform(data)[:,0:NumPCs]
    PCA_Projections_Test = pca.transform(test_data)[:,0:NumPCs]
    return PCA_Projections,PCA_Projections_Test



#return rolling mean of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_mean(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_mean(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'mean' + str(i)})
    return np.array(new_df.astype('float32'))



#return rolling variance of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_var(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_var(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'var' + str(i)})
    return np.array(new_df.astype('float32'))


#return rolling min of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_min(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_min(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'min' + str(i)})
    return np.array(new_df.astype('float32'))


#return rolling max of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_max(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_max(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'max' + str(i)})
    return np.array(new_df.astype('float32'))



#return rolling skew of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_skew(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_skew(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'skew' + str(i)})
    return np.array(new_df.astype('float32'))



#return rolling skew of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_kurt(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_kurt(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'kurt' + str(i)})
    return np.array(new_df.astype('float32'))



#return rolling quantile of each column in a pandas dataframe with a given window and quantile. Returns df of same size. 
def df_rolling_quantile(df,window,quantile):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_quantile(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'Pct'+ quantile + "_" + str(i)})
    return np.array(new_df.astype('float32'))


#BE CAREFUL NOT TO SUPPLY TOO MANY COLUMNS TO THIS FUNCTION. Returns 2^N columns, where N = intitial columns. 
#return rolling pairwise correlation of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_corr(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        list_.append(pd.rolling_corr(df.iloc[0:,i],window,min_periods = 0))
    return pd.concat(list_,1)



#BE CAREFUL NOT TO SUPPLY TOO MANY COLUMNS TO THIS FUNCTION. Returns 2^N columns, where N = intitial columns. 
#return rolling pairwise correlation of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_cov(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        list_.append(rolling_cov(df.iloc[0:,i],window,min_periods = 0))
    return pd.concat(list_,1)




Set Variables and what data will be in model.

In [9]:
#Define Number of Bins with which to separate the data into features based on time progresses since test start. 
Num_Bins = 50

Pct_Variance_Explained = .7
PCA_Rolling_Window = 50 #window with which to use for rolling stats on PCs
Raw_Data_Rolling_Window = 50 #window with which to use for rolling stats on raw data


Train_Feature_List = ['Train_Data','RawData_Rolling_Skewness_Train','RawData_Rolling_Max_Train','RawData_Rolling_Min_Train']

Test_Feature_List = [None]*len(Train_Feature_List)
for i in range(0,len(Train_Feature_List)): 
    print i
    Test_Feature_List[i] = re.sub('Train','Test', Train_Feature_List[i])



0
1
2
3
















This chunk of code will us the Bin_Time function to bin each series into n number of features based on the length of the series. 

In [6]:
Train_Bin_List = []
for i in range (0,len(Train_Array_Lengths)):
    Train_Bin_List.append(Bin_Time(Train_Array_Lengths[i],Num_Bins)) # bin the time based on the sizes of each series. 
Train_Time_Bins = np.concatenate(Train_Bin_List, axis = 0)


Test_Bin_List = []
for i in range (0,len(Test_Array_Lengths)):
    Test_Bin_List.append(Bin_Time(Test_Array_Lengths[i],Num_Bins)) # bin the time based on the sizes of each series. 
Test_Time_Bins = np.concatenate(Test_Bin_List, axis = 0)



In [10]:
Scale_Center = StandardScaler()
Normalized_Train_Data = Scale_Center.fit_transform(np.array(Train_Data))
Normalized_Test_Data = Scale_Center.fit_transform(np.array(Test_Data))


df_Normalized_Train_Data = pd.DataFrame(Normalized_Train_Data)
df_Normalized_Test_Data = pd.DataFrame(Normalized_Test_Data)

PCA_Projections_Train, PCA_Projections_Test = extractFeatures_PCA(df_Normalized_Train_Data,df_Normalized_Test_Data,Pct_Variance_Explained)

PCA_Rolling_Means_Train = df_rolling_mean(pd.DataFrame(PCA_Projections_Train),PCA_Rolling_Window)
PCA_Rolling_Var_Train = df_rolling_var(pd.DataFrame(PCA_Projections_Train),PCA_Rolling_Window)
PCA_Rolling_Skewness_Train = df_rolling_skew(pd.DataFrame(PCA_Projections_Train),PCA_Rolling_Window)
PCA_Rolling_Kurtosis = df_rolling_kurt(pd.DataFrame(PCA_Projections_Train),PCA_Rolling_Window)
PCA_Rolling_Min_Train = df_rolling_min(pd.DataFrame(PCA_Projections_Train),PCA_Rolling_Window)
PCA_Rolling_Max_Train = df_rolling_max(pd.DataFrame(PCA_Projections_Train),PCA_Rolling_Window)


PCA_Rolling_Means_Test = df_rolling_mean(pd.DataFrame(PCA_Projections_Test),PCA_Rolling_Window)
PCA_Rolling_Var_Test = df_rolling_var(pd.DataFrame(PCA_Projections_Test),PCA_Rolling_Window)
PCA_Rolling_Skewness_Test = df_rolling_skew(pd.DataFrame(PCA_Projections_Test),PCA_Rolling_Window)
PCA_Rolling_Kurtosis_Test = df_rolling_kurt(pd.DataFrame(PCA_Projections_Test),PCA_Rolling_Window)
PCA_Rolling_Min_Test = df_rolling_min(pd.DataFrame(PCA_Projections_Test),PCA_Rolling_Window)
PCA_Rolling_Max_Test = df_rolling_max(pd.DataFrame(PCA_Projections_Test),PCA_Rolling_Window)

Feature_Array_Train = np.concatenate((PCA_Projections_Train,PCA_Rolling_Means_Train,PCA_Rolling_Var_Train,PCA_Rolling_Min_Train,PCA_Rolling_Max_Train,Train_Time_Bins),axis=1)
Feature_Array_Test = np.concatenate((PCA_Projections_Test,PCA_Rolling_Means_Test,PCA_Rolling_Var_Test,PCA_Rolling_Min_Test,PCA_Rolling_Max_Test,Test_Time_Bins),axis=1)


391450


In [11]:

RawData_Rolling_Means_Train = df_rolling_mean(pd.DataFrame(Train_Data),Raw_Data_Rolling_Window)
RawData_Rolling_Var_Train = df_rolling_var(pd.DataFrame(Train_Data),Raw_Data_Rolling_Window)
RawData_Rolling_Skewness_Train = df_rolling_skew(pd.DataFrame(Train_Data),Raw_Data_Rolling_Window)
RawData_Rolling_Kurtosis = df_rolling_kurt(pd.DataFrame(Train_Data),Raw_Data_Rolling_Window)
RawData_Rolling_Min_Train = df_rolling_min(pd.DataFrame(Train_Data),Raw_Data_Rolling_Window)
RawData_Rolling_Max_Train = df_rolling_max(pd.DataFrame(Train_Data),Raw_Data_Rolling_Window)


RawData_Rolling_Means_Test = df_rolling_mean(pd.DataFrame(Test_Data),Raw_Data_Rolling_Window)
RawData_Rolling_Var_Test = df_rolling_var(pd.DataFrame(Test_Data),Raw_Data_Rolling_Window)
RawData_Rolling_Skewness_Test = df_rolling_skew(pd.DataFrame(Test_Data),Raw_Data_Rolling_Window)
RawData_Rolling_Kurtosis_Test = df_rolling_kurt(pd.DataFrame(Test_Data),Raw_Data_Rolling_Window)
RawData_Rolling_Min_Test = df_rolling_min(pd.DataFrame(Test_Data),Raw_Data_Rolling_Window)
RawData_Rolling_Max_Test = df_rolling_max(pd.DataFrame(Test_Data),Raw_Data_Rolling_Window)

In [12]:
Feature_Array_Train = np.concatenate([eval(n) for n in Train_Feature_List],axis=1)

Feature_Array_Test = np.concatenate([eval(n) for n in Test_Feature_List],axis=1)



In [13]:
from sklearn import metrics

C_Value = 1
penalty = 'l1'
Convergence_tol = .01

Logistic_Reg = LogisticRegression(C = C_Value, penalty = penalty,tol=Convergence_tol) 

Logistic_Reg.fit(Feature_Array_Train, Train_Labels_HandStart)
HandStart_Predictions = Logistic_Reg.predict(Feature_Array_Test)


# Logistic_Reg.fit(Feature_Array_Train, Train_Labels_FirstDigitTouch)
# FirstDigitTouch_Predictions = Logistic_Reg.predict(Feature_Array_Test)

# Logistic_Reg.fit(Feature_Array_Train, Train_Labels_BothStartLoadPhase)
# BothStartLoadPhase_Predictions = Logistic_Reg.predict(Feature_Array_Test)

# Logistic_Reg.fit(Feature_Array_Train, Train_Labels_LiftOff)
# LiftOff_Predictions = Logistic_Reg.predict(Feature_Array_Test)

# Logistic_Reg.fit(Feature_Array_Train, Train_Labels_Replace)
# Replace_Predictions = Logistic_Reg.predict(Feature_Array_Test)

# Logistic_Reg.fit(Feature_Array_Train, Train_Labels_BothReleased)
# BothReleased_Predictions = Logistic_Reg.predict(Feature_Array_Test)

In [ ]:
# print auc
fpr, tpr, thresholds = metrics.roc_curve(Test_Labels_HandStart, HandStart_Predictions, pos_label=1)
print(metrics.auc(fpr, tpr))

In [32]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.datasets import make_classification


# Create the RFE object and compute a cross-validated score.
LogReg = LogisticRegression(C = C_Value, penalty = penalty,tol=Convergence_tol) 
# The "accuracy" scoring is proportional to the number of correct
# classifications
rfe = RFE(LogReg,10, step=2)
rfe.fit(Feature_Array_Train, Train_Labels_HandStart)

print rfe.estimator_ 



LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l1', random_state=None, tol=0.01)


Look at the accuracy of the model. The first part looks at the overall accuracy when all 6 events are combined into one array. The accuracy is then broken down by the 6 parts. All accuracies are presented as the percent improvement over random. 

# Test Different Models

##Logistic Regession

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#set parameters for Logistic Regression
#Setting them here will allow the optimization of the parameters. We saw in project 2 how much the C-value can affect the results. 
C_Value = 1
penalty = 'l1'
Convergence_tol = .01

#Create a function that trains and runs a logistic regression model. Then prints and returns the AUC score
def predictCategory_LogReg(train_data, train_label, test_data, test_label, Category, C_Value,penalty, Convergence_tol):
    Logistic_Reg = LogisticRegression(C = C_Value, penalty = penalty,tol=Convergence_tol)    
    Logistic_Reg.fit(train_data, train_label)
    print(Category, " Score =", Logistic_Reg.score(test_data, test_label))

    prob = Logistic_Reg.predict_proba(test_data)
    print(Category, "AUC", roc_auc_score(test_label, prob[:,1]))

    #FPR, TPR, thresholds = roc_curve(Test_Labels_HandStart, prob[:,1])
    return roc_auc_score(test_label, prob[:,1])
    
AUC_HandStart = predictCategory_LogReg(Feature_Array_Train,Train_Labels_HandStart,Feature_Array_Test,Test_Labels_HandStart, 'HandStart', C_Value,penalty, Convergence_tol)
AUC_FirstDigitTouch = predictCategory_LogReg(Feature_Array_Train,Train_Labels_FirstDigitTouch,Feature_Array_Test,Test_Labels_FirstDigitTouch, 'FirstDigitTouch', C_Value,penalty, Convergence_tol)
AUC_BothStartLoadPhase = predictCategory_LogReg(Feature_Array_Train,Train_Labels_BothStartLoadPhase,Feature_Array_Test,Test_Labels_BothStartLoadPhase, 'BothStartLoadPhase', C_Value,penalty, Convergence_tol)
AUC_LiftOff = predictCategory_LogReg(Feature_Array_Train,Train_Labels_LiftOff,Feature_Array_Test,Test_Labels_LiftOff, 'LiftOff', C_Value,penalty, Convergence_tol)
AUC_Replace = predictCategory_LogReg(Feature_Array_Train,Train_Labels_Replace,Feature_Array_Test,Test_Labels_Replace, 'Replace', C_Value,penalty, Convergence_tol)
AUC_BothReleased = predictCategory_LogReg(Feature_Array_Train,Train_Labels_BothReleased,Feature_Array_Test,Test_Labels_BothReleased, 'BothReleased', C_Value,penalty, Convergence_tol)

print "Overall Logistic Regression Score = ", np.mean((AUC_HandStart, AUC_FirstDigitTouch,AUC_BothStartLoadPhase,AUC_LiftOff,AUC_Replace,AUC_BothReleased))

In [ ]:

import multiprocessing as mp

Start_Time = time.time()

# Define an output queue
output = mp.Queue()

def Log_Reg_HandStart(output):
    AUC = predictCategory_LogReg(Feature_Array_Train,Train_Labels_HandStart,Feature_Array_Test,Test_Labels_HandStart, 'HandStart', C_Value,penalty, Convergence_tol)
    output.put(AUC)
    
def Log_Reg_FirstDigitTouch(output):    
    AUC = predictCategory_LogReg(Feature_Array_Train,Train_Labels_FirstDigitTouch,Feature_Array_Test,Test_Labels_FirstDigitTouch, 'FirstDigitTouch', C_Value,penalty, Convergence_tol)
    output.put(AUC)
    
def Log_Reg_BothStartLoadPhase(output):       
    AUC = predictCategory_LogReg(Feature_Array_Train,Train_Labels_BothStartLoadPhase,Feature_Array_Test,Test_Labels_BothStartLoadPhase, 'BothStartLoadPhase', C_Value,penalty, Convergence_tol)
    output.put(AUC)
    
def Log_Reg_LiftOff(output):       
    AUC = predictCategory_LogReg(Feature_Array_Train,Train_Labels_LiftOff,Feature_Array_Test,Test_Labels_LiftOff, 'LiftOff', C_Value,penalty, Convergence_tol)
    output.put(AUC)
    
def Log_Reg_Replace(output):     
    AUC = predictCategory_LogReg(Feature_Array_Train,Train_Labels_Replace,Feature_Array_Test,Test_Labels_Replace, 'Replace', C_Value,penalty, Convergence_tol)
    output.put(AUC)
    
def Log_Reg_BothReleased(output):    
    AUC = predictCategory_LogReg(Feature_Array_Train,Train_Labels_BothReleased,Feature_Array_Test,Test_Labels_BothReleased, 'BothReleased', C_Value,penalty, Convergence_tol)
    output.put(AUC)
    
# Setup a list of processes that we want to run
processes = [mp.Process(target=Log_Reg_HandStart)
            ,mp.Process(target=Log_Reg_FirstDigitTouch)
            ,mp.Process(target=Log_Reg_BothStartLoadPhase)
            ,mp.Process(target=Log_Reg_LiftOff)
            ,mp.Process(target=Log_Reg_Replace)
            ,mp.Process(target=Log_Reg_BothReleased)]

print processes
# Run processes
for p in processes:
    p.start()
 


for p in processes:
    p.join()
 
# Get process results from the output queue
results = [output.get() for p in processes]
    
print "Overall Logistic Regression Score = ", np.mean((AUC_HandStart, AUC_FirstDigitTouch,AUC_BothStartLoadPhase,AUC_LiftOff,AUC_Replace,AUC_BothReleased))

if __name__ == '__main__':
    print "IFFFFF"
    p = Process(target=Log_Reg_Summary)
    p.start()
    p.join()

# AUC_HandStart = predictCategory_LogReg(Feature_Array_Train,Train_Labels_HandStart,Feature_Array_Test,Test_Labels_HandStart, 'HandStart', C_Value,penalty, Convergence_tol)
# AUC_FirstDigitTouch = predictCategory_LogReg(Feature_Array_Train,Train_Labels_FirstDigitTouch,Feature_Array_Test,Test_Labels_FirstDigitTouch, 'FirstDigitTouch', C_Value,penalty, Convergence_tol)
# AUC_BothStartLoadPhase = predictCategory_LogReg(Feature_Array_Train,Train_Labels_BothStartLoadPhase,Feature_Array_Test,Test_Labels_BothStartLoadPhase, 'BothStartLoadPhase', C_Value,penalty, Convergence_tol)

print time.time()-Start_Time

In [28]:

from multiprocessing import Process
    
Start_Time = time.time()
def runInParallel(*fns):
    proc = []
    for fn in fns:
        p = Process(target=fn)
        p.start()
        #proc.append(p)
    #for p in proc:
        #p.join()

output = runInParallel(predictCategory_LogReg(Feature_Array_Train,Train_Labels_HandStart,Feature_Array_Test,Test_Labels_HandStart, 'HandStart', C_Value,penalty, Convergence_tol)
, predictCategory_LogReg(Feature_Array_Train,Train_Labels_BothStartLoadPhase,Feature_Array_Test,Test_Labels_BothStartLoadPhase, 'BothStartLoadPhase', C_Value,penalty, Convergence_tol)
, predictCategory_LogReg(Feature_Array_Train,Train_Labels_BothStartLoadPhase,Feature_Array_Test,Test_Labels_BothStartLoadPhase, 'BothStartLoadPhase', C_Value,penalty, Convergence_tol)
)
print output
print time.time()-Start_Time

('HandStart', ' Score =', 0.98028619482202428)
('HandStart', 'AUC', 0.73293521880342571)
('BothStartLoadPhase', ' Score =', 0.98018509838521417)
('BothStartLoadPhase', 'AUC', 0.71070132577101419)
('BothStartLoadPhase', ' Score =', 0.98019888426296098)
('BothStartLoadPhase', 'AUC', 0.71155041096067695)
None
43.861000061


## SVM Model

This is set up to get started. It has not been tested. MM 08/04/2015 11:30 A.M.

In [ ]:
#set parameters for SVM
#Setting them here will allow the optimization of the parameters. We saw in project 2 how much the C-value can affect the results. 
C_Value = 1
kernel = 'rbf'
degree = 3

#Create a function that trains and runs a model. Then prints and returns the AUC score
def predictCategory_SVM(train_data, train_label, test_data, test_label, Category, C_Value,kernel, degree):
    SVM = SVC(C = C_Value, kernel = kernel,degree=degree)    
    SVC.fit(train_data, train_label)

    prob = SVC.predict_proba(test_data)
    print(Category, "AUC", roc_auc_score(test_label, prob[:,1]))

    #FPR, TPR, thresholds = roc_curve(Test_Labels_HandStart, prob[:,1])
    return roc_auc_score(test_label, prob[:,1])
    
AUC_HandStart = predictCategory_SVM(Feature_Array_Train,Train_Labels_HandStart,Feature_Array_Test,Test_Labels_HandStart, 'HandStart', Category, C_Value,kernel, degree)
AUC_FirstDigitTouch = predictCategory_SVM(Feature_Array_Train,Train_Labels_FirstDigitTouch,Feature_Array_Test,Test_Labels_FirstDigitTouch, 'FirstDigitTouch', Category, C_Value,kernel, degree)
AUC_BothStartLoadPhase = predictCategory_SVM(Feature_Array_Train,Train_Labels_BothStartLoadPhase,Feature_Array_Test,Test_Labels_BothStartLoadPhase, 'BothStartLoadPhase', Category, C_Value,kernel, degree)
AUC_LiftOff = predictCategory_SVM(Feature_Array_Train,Train_Labels_LiftOff,Feature_Array_Test,Test_Labels_LiftOff, 'LiftOff', Category, C_Value,kernel, degree)
AUC_Replace = predictCategory_SVM(Feature_Array_Train,Train_Labels_Replace,Feature_Array_Test,Test_Labels_Replace, 'Replace', Category, C_Value,kernel, degree)
AUC_BothReleased = predictCategory_SVM(Feature_Array_Train,Train_Labels_BothReleased,Feature_Array_Test,Test_Labels_BothReleased, 'BothReleased', Category, C_Value,kernel, degree


print "Overall SVM Score = ", np.mean(AUC_HandStart, AUC_FirstDigitTouch,AUC_BothStartLoadPhase,AUC_LiftOff,AUC_Replace,AUC_BothReleased)

##Neural Network

In [ ]:
#set parameters for Neural Network model
#Setting them here will allow the optimization of the parameters. We saw in project 2 how much the C-value can affect the results. 
n_components = 10
learning_rate = .1
batch_size = 20000
n_iter =1


#Create a function that trains and runs a model. Then prints and returns the AUC score
def predictCategory_Neural_Network(train_data, train_label, test_data, test_label, Category, C_Value,kernel, degree):
    SVM = SVC(C = C_Value, kernel = kernel,degree=degree)    
    SVC.fit(train_data, train_label)

    prob = SVC.predict_proba(test_data)
    print(Category, "AUC", roc_auc_score(test_label, prob[:,1]))

    #FPR, TPR, thresholds = roc_curve(Test_Labels_HandStart, prob[:,1])
    return roc_auc_score(test_label, prob[:,1])
    
AUC_HandStart = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_HandStart,Feature_Array_Test,Test_Labels_HandStart, 'HandStart', Category, C_Value,kernel, degree)
AUC_FirstDigitTouch = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_FirstDigitTouch,Feature_Array_Test,Test_Labels_FirstDigitTouch, 'FirstDigitTouch', Category, C_Value,kernel, degree)
AUC_BothStartLoadPhase = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_BothStartLoadPhase,Feature_Array_Test,Test_Labels_BothStartLoadPhase, 'BothStartLoadPhase', Category, C_Value,kernel, degree)
AUC_LiftOff = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_LiftOff,Feature_Array_Test,Test_Labels_LiftOff, 'LiftOff', Category, C_Value,kernel, degree)
AUC_Replace = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_Replace,Feature_Array_Test,Test_Labels_Replace, 'Replace', Category, C_Value,kernel, degree)
AUC_BothReleased = predictCategory_Neural_Network(Feature_Array_Train,Train_Labels_BothReleased,Feature_Array_Test,Test_Labels_BothReleased, 'BothReleased', Category, C_Value,kernel, degree


print "Overall SVM Score = ", np.mean(AUC_HandStart, AUC_FirstDigitTouch,AUC_BothStartLoadPhase,AUC_LiftOff,AUC_Replace,AUC_BothReleased)


## Decision Trees and Random Forest